## Mount the Drive, and Change to Google Drive Folder

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

# %cd /content/drive/MyDrive/MSc.-Dissertations/1/Files
# %ls

## Import Libraries

In [11]:
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, utils, losses
from keras.wrappers import scikit_learn
from keras.models import *
from keras.layers import *
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from keras.utils import np_utils

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix

import random
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

## Count the Number of Files, and Take Random Samples from the Image Files

In [2]:
# !ls street_view
# count how many files and write the filenames into a file
# !ls street_view -1 | wc -l 
# !ls street_view/*.jpg > flist.txt
flist = list(pd.read_csv('flist.txt', header = None)[0])

# Set seed so sample is reproducible 
# random.seed(99)  # set this to an integer value!!!
# nsamp = 100
# flist_sub = random.sample(flist, nsamp)
# flist = flist_sub

# print(flist)

## Overview of the `properties` Dataset

Read the `properties` dataset first, and make sure that `property type` is a categorical variable.

In [3]:
properties = pd.read_csv('properties.csv')
properties.propertyType = properties.propertyType.astype('category')
properties.head()

,Unnamed: 0,address,propertyType,bedrooms,detailUrl,location_lat,location_lng,property_id
0,0,"12, Gorsey Brigg, Dronfield Woodhouse, Dronfie...",Terraced,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29986,-1.49446,60d9dd15-c5a0-4d9c-a341-a1d47add49d5
1,0,"5, Highgate Lane, Dronfield, Derbyshire S18 1UB",Detached,4.0,https://www.rightmove.co.uk/house-prices/detai...,53.29135,-1.45975,4a586e80-181a-4b82-b5c3-2d789436bb14
2,0,"125, Gosforth Lane, Dronfield, Derbyshire S18 1RB",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29763,-1.47573,93680b6c-237e-44d3-8f40-959a14b80cad
3,0,"80, Shakespeare Crescent, Dronfield, Derbyshir...",Detached,3.0,https://www.rightmove.co.uk/house-prices/detai...,53.29259,-1.45644,5d49758b-f148-4d06-bbae-3eb23f5c68fb
4,0,"21, Gainsborough Road, Dronfield, Derbyshire S...",Detached,NaN,https://www.rightmove.co.uk/house-prices/detai...,53.29740,-1.48503,4645f5eb-de7c-474f-8d7e-b59fa8c55f19


Basic information of the dataset is shown as follows.

In [ ]:
properties.info()

Descriptive statistics of continuous variables are shown as follows.

In [ ]:
properties.describe()

Frequencies of each level of the variable `property type` are obtained as follows.

In [ ]:
properties.propertyType.value_counts()

## A Subset of the `properties` Dataset

As random samples of images have been obtained previously, a subset of the whole `properties` dataset could hence be formulated by selecting the rows of the whole `properties` dataset corresponding to the selected samples.

In [4]:
flist_id = list(map(lambda string: string[16 : -4], flist))
properties_sub = pd.DataFrame(properties.loc[properties['property_id'].isin(flist_id)])
properties_sub = properties_sub.drop_duplicates(['address', 'detailUrl', 'location_lat', 'location_lng'])
flist_id = list(properties_sub.property_id)
flist_new = []
temp = list(map(lambda x: flist_new.append(f'street_view/gsv_{x}.jpg'), flist_id))

Dictionaries are created to link `property ID` with our variables of interest.

In [ ]:
# dic_propID_imgArray = dict(zip(flist_id, list(map(lambda x: np.array(Image.open(x)), flist_new)))) # dictionary of RGB values in each pixel
# dic_propID_propType = dict(zip(flist_id, properties_sub.propertyType)) # dictionary of property types

The original data should be splitted into training and testing sets, and the testing set contains 30% of the original data.

In [5]:
Img_array_train, Img_array_test, propertyType_train, propertyType_test = train_test_split(
    np.array(list(map(lambda x: np.array(Image.open(x)), flist_new))), # RGB values in each pixel
    np.array(properties_sub.propertyType), # property types
    test_size = 0.3)

For categorical variables, one-hot encoder is introduced.

In [6]:
dummy_propertyType_train = pd.get_dummies(propertyType_train)
dummy_propertyType_test = pd.get_dummies(propertyType_test)
propertyType_test_fac = np.argmax(np.array(dummy_propertyType_test), axis = 1) 

## Multi-Class Classification Using Neural Network

### Multi-Layer Perceptron (MLP) model

In [7]:
def mlp(output_dim):

    '''
    Creates a multi-layer perceptron neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Flatten())
    # model.add(Dense(100, activation = tf.nn.leaky_relu))
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [15]:
MLP = mlp(dummy_propertyType_train.shape[1])
MLP.fit(Img_array_train, dummy_propertyType_train, epochs = 100, batch_size = 50)

Epoch 1/100
7/7 [==============================] - 1s 114ms/step - loss: 147.5202 - accuracy: 0.1836
Epoch 2/100
7/7 [==============================] - 1s 103ms/step - loss: 157.5188 - accuracy: 0.2492
Epoch 3/100
7/7 [==============================] - 1s 120ms/step - loss: 85.5925 - accuracy: 0.2033
Epoch 4/100
7/7 [==============================] - 1s 103ms/step - loss: 59.2720 - accuracy: 0.2984
Epoch 5/100
7/7 [==============================] - 1s 103ms/step - loss: 63.3905 - accuracy: 0.2820
Epoch 6/100
7/7 [==============================] - 1s 100ms/step - loss: 27.2860 - accuracy: 0.3377
Epoch 7/100
7/7 [==============================] - 1s 99ms/step - loss: 20.2335 - accuracy: 0.3770
Epoch 8/100
7/7 [==============================] - 1s 100ms/step - loss: 13.5945 - accuracy: 0.3803
Epoch 9/100
7/7 [==============================] - 1s 103ms/step - loss: 12.0764 - accuracy: 0.4623
Epoch 10/100
7/7 [==============================] - 1s 97ms/step - loss: 10.2137 - accuracy: 0.4492

In [ ]:
propertyType_test_pred = np.argmax(MLP.predict(Img_array_test), axis = 1) 

In [ ]:
confusion_matrix(propertyType_test_fac, propertyType_test_pred)

## Convolutional Neural Network

In [ ]:
def cnn(output_dim):

    '''
    Creates a convolutional neural network model without hidden layers.

    Parameter:
    output_dim (int): The number of output classes.
    
    Returns:
    A compiled Keras model.
    '''

    model = Sequential()
    model.add(Rescaling(1. / 255))
    model.add(Conv2D(16, 3, padding = 'same', activation = tf.nn.relu))
    model.add(MaxPooling2D())   
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(output_dim, activation = tf.nn.softmax))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
CNN = cnn(dummy_propertyType_train.shape[1])
CNN.fit(Img_array_train, dummy_propertyType_train, epochs = 15, batch_size = 50)